In [1]:
import os
import json
import glob
import pandas as pd
from tqdm import tqdm  # Import tqdm

def reviews_to_dataframe(json_data):
    all_reviews_df = pd.DataFrame()
    
    for product_data in json_data:
        if 'body' not in product_data or 'reviews' not in product_data['body']:
            continue
        reviews_data = product_data['body'].get('reviews', [])
        product_name = product_data['body'].get('name', 'Unknown Product')
        
        if not reviews_data:
            continue
        
        reviews_df = pd.DataFrame.from_records(reviews_data)

        columns_to_include = [
            "reviewerName", 
            "reviewerLink", 
            "reviewRating", 
            "reviewDate", 
            "reviewTitle", 
            "reviewText", 
            "reviewVotes", 
            "reviewVerifiedPurchase"
        ]
        existing_columns = [col for col in columns_to_include if col in reviews_df.columns]
        
        if not existing_columns:
            continue
        
        reviews_df = reviews_df[existing_columns]
        
        if 'reviewRating' in existing_columns:
            reviews_df['reviewRating'] = reviews_df['reviewRating'].str.extract(r'(\d+\.\d+)').astype(float)
        if 'reviewDate' in existing_columns:
            reviews_df[['Location', 'Date']] = reviews_df['reviewDate'].str.extract(r'Reviewed in the ([\w\s]+) on (.+)$')
        if 'reviewVotes' in existing_columns:
            reviews_df['reviewVotes'] = reviews_df['reviewVotes'].str.extract(r'(\d+)').fillna(0).astype(int)
        if 'reviewerLink' in existing_columns:
            reviews_df['reviewerID'] = reviews_df['reviewerLink'].str.extract(r'.*amzn1.account.([^/]+)')
        
        for col in ['reviewDate', 'reviewerLink']:
            if col in reviews_df.columns:
                reviews_df.drop(columns=[col], inplace=True)
        
        reviews_df['ProductName'] = product_name
        
        all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)
    
    return all_reviews_df

In [2]:
base_dir = '..\amazon'

all_json_data = []

for root, dirs, files in os.walk(base_dir):
    for dir in dirs:
        items_path = os.path.join(root, dir, 'items')
        
        if os.path.exists(items_path):
            json_files = glob.glob(os.path.join(items_path, '*.json'))
            
            for json_file in tqdm(json_files, desc=f'Loading JSON Files in {dir}'):
                try:
                    with open(json_file, "r") as f:
                        all_json_data.append(json.load(f))
                except json.JSONDecodeError:
                    print(f"Error loading JSON from file {json_file}: file is empty or not a valid JSON.")
                except Exception as e:
                    print(f"Unexpected error loading JSON from file {json_file}: {e}")

all_reviews_df = reviews_to_dataframe(all_json_data)
all_reviews_df.head()

""


In [3]:
base_dir = '../amazon'

all_json_data = []

for root, dirs, files in os.walk(base_dir):
    for dir in tqdm(dirs, desc='Directories Progress'):
        items_path = os.path.join(root, dir, 'items')
        if os.path.exists(items_path):
            json_files = glob.glob(os.path.join(items_path, '*.json'))
            for json_file in tqdm(json_files, desc=f'Loading {dir}'):
                try:
                    with open(json_file, 'r') as f:
                        data = json.load(f)
                        all_json_data.append(data)
                except json.JSONDecodeError:
                    print(f'Invalid JSON in file: {json_file}')
                except Exception as e:
                    print(f'Error reading file {json_file}: {e}')

print(f"Total JSON files loaded: {len(all_json_data)}")

Loading air fryer:   5%|███▏                                                         | 11/211 [00:00<00:01, 104.74it/s]


Directories Progress:   5%|██▊                                                         | 8/171 [00:13<04:37,  1.70s/it]


Loading bedding: 100%|██████████████████████████████████████████████████████████████| 246/246 [00:02<00:00, 114.14it/s]


Loading bookshelf:  82%|█████████████████████████████████████████████████▍          | 225/273 [00:01<00:00, 124.00it/s]


Loading cables:   0%|                                                                          | 0/241 [00:00<?, ?it/s]


...

Directories Progress: 100%|██████████████████████████████████████████████████████████| 171/171 [05:13<00:00,  1.83s/it]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|███████

Directories Progress: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 999.36it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 998.17it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, 

Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|███████

Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 999.12it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, ?it/s]
Directories Progress: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Directories Progress: 0it [00:00, 

Total JSON files loaded: 37769


In [5]:
if all_json_data:
    print(json.dumps(all_json_data[0], indent=2, ensure_ascii=False))
else:
    print("No data loaded. Check the JSON file paths and their contents.")

{
  "remaining_requests": 5090,
  "original_status": 200,
  "pc_status": 200,
  "url": "https://www.amazon.com/Trooper-6-Inch-Scale-Mandalorian-Collectible-F4066/dp/B001F30182",
  "body": {
    "name": "STAR WARS The Black Series Dark Trooper Toy 6-Inch-Scale The Mandalorian Collectible Action Figure, Toys for Kids Ages 4 and Up",
    "price": "$16.77",
    "originalPrice": "$33.99",
    "rawPrice": 16.77,
    "currency": "$",
    "canonicalUrl": "https://www.amazon.com/Trooper-6-Inch-Scale-Mandalorian-Collectible-F4066/dp/B001F30182",
    "soldBy": {
      "sellerID": "AD6RFAAQRI8WI",
      "name": "Firefly Buys",
      "link": "https://www.amazon.com/gp/help/seller/at-a-glance.html/ref=dp_merchant_link?ie=UTF8&seller=AD6RFAAQRI8WI&asin=B001F30182&ref_=dp_merchant_link&isAmazonFulfilled=1"
    },
    "deliveredBy": "",
    "parentAsin": "",
    "asinVariationValues": [],
    "isPrime": true,
    "inStock": true,
    "maximumQuantity": "30",
    "stockDetail": "In Stock",
    "couponIn

In [7]:
def reviews_to_dataframe(json_data):
    all_reviews_df = pd.DataFrame()

    for product_data in tqdm(json_data, desc="Processing Items"):
        product_name = product_data.get('body', {}).get('name', 'Unknown Product')

        if 'reviews' in product_data.get('body', {}):
            reviews_data = product_data['body']['reviews']

            reviews_list = []

            for review in reviews_data:
                review_dict = {
                    'reviewerName': review.get('reviewerName', ''),
                    'reviewRating': review.get('reviewRating', '').split(' out of ')[0],
                    'reviewDate': review.get('reviewDate', ''),
                    'reviewTitle': review.get('reviewTitle', ''),
                    'reviewText': review.get('reviewText', ''),
                    'reviewVotes': review.get('reviewVotes', '').split()[0] if review.get('reviewVotes', '') else '0', 
                    'reviewVerifiedPurchase': review.get('reviewVerifiedPurchase', False),
                    'ProductName': product_name
                }
                reviews_list.append(review_dict)

            reviews_df = pd.DataFrame(reviews_list)

            all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)

    return all_reviews_df

all_reviews_df = reviews_to_dataframe(all_json_data)
all_reviews_df.head()

Processing Items: 100%|████████████████████████████████████████████████████████| 37769/37769 [1:18:38<00:00,  8.01it/s]


,reviewerName,reviewRating,reviewDate,reviewTitle,reviewText,reviewVotes,reviewVerifiedPurchase,ProductName
0,TK-815TK-815,5.0,"Reviewed in the United States on September 21,...","5.0 out of 5 stars Fantastic War Droid, Not So...",I am impressed with this figure. The sculpt is...,13,True,STAR WARS The Black Series Dark Trooper Toy 6-...
1,vyeranos,5.0,"Reviewed in the United States on January 13, 2024",5.0 out of 5 stars Good addition to the collec...,Delivered on time and at a good price. My than...,0,True,STAR WARS The Black Series Dark Trooper Toy 6-...
2,Marty,5.0,"Reviewed in the United States on November 24, ...",5.0 out of 5 stars Detailed,These figures look pretty detailed,0,True,STAR WARS The Black Series Dark Trooper Toy 6-...
3,Kati,5.0,"Reviewed in the United States on January 2, 2024",5.0 out of 5 stars Perfect,"Perfect condition, quick delivery",0,True,STAR WARS The Black Series Dark Trooper Toy 6-...
4,Lochlan LongstriderLochlan Longstrider,4.0,"Reviewed in the United States on September 18,...",4.0 out of 5 stars Needs a stand,"Cool dark droid; although, he really doesn't l...",0,True,STAR WARS The Black Series Dark Trooper Toy 6-...


In [9]:
all_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325340 entries, 0 to 325339
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   reviewerName            325340 non-null  object
 1   reviewRating            325340 non-null  object
 2   reviewDate              325340 non-null  object
 3   reviewTitle             325340 non-null  object
 4   reviewText              325340 non-null  object
 5   reviewVotes             325340 non-null  object
 6   reviewVerifiedPurchase  325340 non-null  bool  
 7   ProductName             325340 non-null  object
dtypes: bool(1), object(7)
memory usage: 17.7+ MB


In [13]:
all_reviews_df.to_csv('All_Reviewers_V0.csv', index=False)

In [15]:
import re

def reviews_to_dataframe(json_data):
    all_reviews_df = pd.DataFrame()

    for product_data in tqdm(json_data, desc="Processing Items"):
        product_name = product_data.get('body', {}).get('name', 'Unknown Product')

        if 'reviews' in product_data.get('body', {}):
            reviews_data = product_data['body']['reviews']

            reviews_list = []

            for review in reviews_data:
                review_rating = re.search(r'(\d+\.\d+|\d+)', review.get('reviewRating', ''))
                review_rating = float(review_rating.group(1)) if review_rating else None

                review_date_match = re.search(r'Reviewed in the ([\w\s]+) on (.+)$', review.get('reviewDate', ''))
                location = review_date_match.group(1) if review_date_match else ''
                review_date = review_date_match.group(2) if review_date_match else ''

                review_votes = re.search(r'(\d+)', review.get('reviewVotes', ''))
                review_votes = int(review_votes.group(1)) if review_votes else 0

                reviewer_id_match = re.search(r'.*amzn1.account.([^/]+)', review.get('reviewerLink', ''))
                reviewer_id = reviewer_id_match.group(1) if reviewer_id_match else ''

                review_dict = {
                    'reviewerName': review.get('reviewerName', ''),
                    'reviewRating': review_rating,
                    'Location': location,
                    'Date': review_date,
                    'reviewTitle': review.get('reviewTitle', ''),
                    'reviewText': review.get('reviewText', ''),
                    'reviewVotes': review_votes,
                    'reviewVerifiedPurchase': review.get('reviewVerifiedPurchase', False),
                    'reviewerID': reviewer_id,
                    'ProductName': product_name
                }
                reviews_list.append(review_dict)

            reviews_df = pd.DataFrame(reviews_list)

            all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)

    return all_reviews_df

all_reviews_df_2 = reviews_to_dataframe(all_json_data)
all_reviews_df_2.head()

Processing Items: 100%|████████████████████████████████████████████████████████| 37769/37769 [1:21:53<00:00,  7.69it/s]


,reviewerName,reviewRating,Location,Date,reviewTitle,reviewText,reviewVotes,reviewVerifiedPurchase,reviewerID,ProductName
0,TK-815TK-815,5.0,United States,"September 21, 2022","5.0 out of 5 stars Fantastic War Droid, Not So...",I am impressed with this figure. The sculpt is...,13,True,AH4NM3KGMDME7SKRSJD73TIBABIQ,STAR WARS The Black Series Dark Trooper Toy 6-...
1,vyeranos,5.0,United States,"January 13, 2024",5.0 out of 5 stars Good addition to the collec...,Delivered on time and at a good price. My than...,0,True,AESWEZHPRRYTZRW2FBIJSXEKGTFA,STAR WARS The Black Series Dark Trooper Toy 6-...
2,Marty,5.0,United States,"November 24, 2023",5.0 out of 5 stars Detailed,These figures look pretty detailed,0,True,AHGIPPEB6HKSXNPB6Y3DLBKCVRHQ,STAR WARS The Black Series Dark Trooper Toy 6-...
3,Kati,5.0,United States,"January 2, 2024",5.0 out of 5 stars Perfect,"Perfect condition, quick delivery",0,True,AHWQ73WR7MVGBKDB5YE6C4WDPNJA,STAR WARS The Black Series Dark Trooper Toy 6-...
4,Lochlan LongstriderLochlan Longstrider,4.0,United States,"September 18, 2023",4.0 out of 5 stars Needs a stand,"Cool dark droid; although, he really doesn't l...",0,True,AHSHUIZHSPWYY3OD3PI56MSBKZ4Q,STAR WARS The Black Series Dark Trooper Toy 6-...


In [19]:
all_reviews_df_2.to_csv('All_Reviewers_V1.csv', index=False)

In [20]:
def reviews_to_dataframe(json_data, folder_names):
    all_reviews_df = pd.DataFrame()

    for product_data, folder_name in tqdm(zip(json_data, folder_names), desc="Processing Items", total=len(json_data)):
        product_name = product_data.get('body', {}).get('name', 'Unknown Product')

        if 'reviews' in product_data.get('body', {}):
            reviews_data = product_data['body']['reviews']
            reviews_list = []

            for review in reviews_data:
                review_rating = re.search(r'(\d+\.\d+|\d+)', review.get('reviewRating', ''))
                review_rating = float(review_rating.group(1)) if review_rating else None

                review_date_match = re.search(r'Reviewed in the ([\w\s]+) on (.+)$', review.get('reviewDate', ''))
                location = review_date_match.group(1) if review_date_match else ''
                review_date = review_date_match.group(2) if review_date_match else ''

                review_votes = re.search(r'(\d+)', review.get('reviewVotes', ''))
                review_votes = int(review_votes.group(1)) if review_votes else 0

                reviewer_id_match = re.search(r'.*amzn1.account.([^/]+)', review.get('reviewerLink', ''))
                reviewer_id = reviewer_id_match.group(1) if reviewer_id_match else ''

                review_dict = {
                    'reviewerName': review.get('reviewerName', ''),
                    'reviewRating': review_rating,
                    'Location': location,
                    'Date': review_date,
                    'reviewTitle': review.get('reviewTitle', ''),
                    'reviewText': review.get('reviewText', ''),
                    'reviewVotes': review_votes,
                    'reviewVerifiedPurchase': review.get('reviewVerifiedPurchase', False),
                    'reviewerID': reviewer_id,
                    'ProductName': product_name,
                    'FolderName': folder_name
                }
                reviews_list.append(review_dict)

            reviews_df = pd.DataFrame(reviews_list)
            all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)

    return all_reviews_df


In [21]:
all_json_data = []
folder_names = []

for root, dirs, files in os.walk(base_dir):
    for dir in dirs:
        items_path = os.path.join(root, dir, 'items')
        if os.path.exists(items_path):
            json_files = glob.glob(os.path.join(items_path, '*.json'))
            for json_file in json_files:
                with open(json_file, 'r') as f:
                    all_json_data.append(json.load(f))
                    folder_names.append(dir)

all_reviews_df_3 = reviews_to_dataframe(all_json_data, folder_names)
all_reviews_df_3.head()

Processing Items: 100%|████████████████████████████████████████████████████████| 37769/37769 [1:57:08<00:00,  5.37it/s]


,reviewerName,reviewRating,Location,Date,reviewTitle,reviewText,reviewVotes,reviewVerifiedPurchase,reviewerID,ProductName,FolderName
0,TK-815TK-815,5.0,United States,"September 21, 2022","5.0 out of 5 stars Fantastic War Droid, Not So...",I am impressed with this figure. The sculpt is...,13,True,AH4NM3KGMDME7SKRSJD73TIBABIQ,STAR WARS The Black Series Dark Trooper Toy 6-...,action figures
1,vyeranos,5.0,United States,"January 13, 2024",5.0 out of 5 stars Good addition to the collec...,Delivered on time and at a good price. My than...,0,True,AESWEZHPRRYTZRW2FBIJSXEKGTFA,STAR WARS The Black Series Dark Trooper Toy 6-...,action figures
2,Marty,5.0,United States,"November 24, 2023",5.0 out of 5 stars Detailed,These figures look pretty detailed,0,True,AHGIPPEB6HKSXNPB6Y3DLBKCVRHQ,STAR WARS The Black Series Dark Trooper Toy 6-...,action figures
3,Kati,5.0,United States,"January 2, 2024",5.0 out of 5 stars Perfect,"Perfect condition, quick delivery",0,True,AHWQ73WR7MVGBKDB5YE6C4WDPNJA,STAR WARS The Black Series Dark Trooper Toy 6-...,action figures
4,Lochlan LongstriderLochlan Longstrider,4.0,United States,"September 18, 2023",4.0 out of 5 stars Needs a stand,"Cool dark droid; although, he really doesn't l...",0,True,AHSHUIZHSPWYY3OD3PI56MSBKZ4Q,STAR WARS The Black Series Dark Trooper Toy 6-...,action figures


In [22]:
all_reviews_df_3.to_csv('All_Reviewers_V2.csv', index=False)